In [1]:

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
uid_train = pd.read_csv('../input/mydata/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('../input/mydata/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_train = pd.read_csv('../input/mydata/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_train = pd.read_csv('../input/mydata/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
voice_test = pd.read_csv('../input/mydata2/voice_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_test = pd.read_csv('../input/mydata2/sms_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_test = pd.read_csv('../input/mydata2/wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [4]:
uid_test = pd.DataFrame({'uid':pd.unique(wa_test['uid'])})
uid_test.to_csv('uid_test_a.txt',index=None)

In [5]:
voice = pd.concat([voice_train,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test],axis=0)

##  baseline 

### 通话记录

In [6]:
voice_opp_num = voice.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('voice_opp_num_').reset_index()

voice_opp_head=voice.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('voice_opp_head_').reset_index()

voice_opp_len=voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').reset_index().fillna(0)

voice_call_type = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').reset_index().fillna(0)

voice_in_out = voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out_').reset_index().fillna(0)
# voice_end_time = voice.groupby(['uid'])['end_time'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('end_time_').reset_index().fillna(0)
# voice_start_time = voice.groupby(['uid'])['start_time'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('start_time_').reset_index().fillna(0)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
voice.start_time= voice.start_time.astype(np.int32) 
voice.end_time= voice.end_time.astype(np.int32) 
sms.start_time= sms.start_time.astype(np.int32) 

# voice_gap_time = voice.groupby(['uid'])['start_time'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_gap_time_').reset_index()

In [34]:
# voice_gap_time = voice.groupby(['uid'])['start_time'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_gap_time_').reset_index()
# voice_gap_end_time = voice.groupby(['uid'])['end_time'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_gap_time_').reset_index()

In [8]:
voice_start_hour_count= voice.groupby(['uid'])['start_time'].agg({'unique_count': lambda x: len(pd.unique((x//10000)%100)),'count':'count'}).add_prefix('voice_start_hour_count_').reset_index()
voice_end_hour_count= voice.groupby(['uid'])['end_time'].agg({'unique_count': lambda x: len(pd.unique((x//10000)%100)),'count1':'count'}).add_prefix('voice_end_hour_count_').reset_index()
voice_start_day_count= voice.groupby(['uid'])['start_time'].agg({'unique_count': lambda x: len(pd.unique(x//1000000)),'count2':'count'}).add_prefix('voice_start_day_count_').reset_index()
voice_end_day_count= voice.groupby(['uid'])['end_time'].agg({'unique_count': lambda x: len(pd.unique(x//1000000)),'count3':'count'}).add_prefix('voice_end_day_count_').reset_index()
voice_early_hour= voice.groupby(['uid'])['start_time'].agg({'unique_count': lambda x: min(pd.unique((x//10000)%100)),'count4':'count'}).add_prefix('voice_early_hour_count_').reset_index()
voice_late_hour= voice.groupby(['uid'])['start_time'].agg({'unique_count': lambda x: max(pd.unique((x//10000)%100)),'count5':'count'}).add_prefix('voice_late_hour_count_').reset_index()
voice_end_early_hour= voice.groupby(['uid'])['end_time'].agg({'unique_count': lambda x: min(pd.unique((x//10000)%100)),'count6':'count'}).add_prefix('voice_early_hour_count_').reset_index()
voice_end_late_hour= voice.groupby(['uid'])['end_time'].agg({'unique_count': lambda x: max(pd.unique((x//10000)%100)),'count7':'count'}).add_prefix('voice_late_hour_count_').reset_index()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprec

## 这有一些参考的特征

In [ ]:
def time_gap(start,end):
    if pd.isnull(start):
        return np.nan
    end_day = int(str(end)[0:2])
    start_day = int(str(start)[0:2])
    day_gap = (end_day-start_day)*86400
    
    
    end_hour = int(str(end)[2:4])
    start_hour = int(str(start)[2:4])
    hour_gap = (end_hour-start_hour)*3600
    
    
    
    end_min = int(str(end)[4:6])
    start_min = int(str(start)[4:6])
    min_gap = (end_min-start_min)*60
    
    
    end_sec = int(str(end)[6:8])
    start_sec = int(str(start)[6:8])
    sec_gap = (end_sec-start_sec)
    
    
    return day_gap+hour_gap+min_gap+sec_gap
    

In [ ]:
## 通话时长统计
voice['gap_time']=voice[['start_time','end_time']].apply(lambda x: time_gap(x[0],x[1]),axis=1)

voice_gap_time=voice.groupby(['uid'])['gap_time'].agg(['std','max','min','median','mean','sum',np.ptp]).add_prefix('voice_gap_time_').reset_index()





In [9]:
voice_call_type_opp_num_unique=voice.groupby(['uid','call_type'])['opp_num'].agg(lambda x: len(pd.unique(x))).unstack().add_prefix('voice_call_type_opp_num_unique_').reset_index().fillna(0)

voice_in_out_opp_num_unique = voice.groupby(['uid','in_out'])['opp_num'].agg(lambda x: len(pd.unique(x))).unstack().add_prefix('voice_in_out_opp_num_unique_').reset_index().fillna(0)



## 短信记录

In [186]:
sms_opp_num = sms.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_opp_num_').reset_index()

sms_opp_head=sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('sms_opp_head_').reset_index()

sms_opp_len=sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_opp_len_').reset_index().fillna(0)
# sms_start_time = sms.groupby(['uid'])['start_time'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_start_time_').reset_index()

sms_in_out = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)


In [187]:
# sms_gap_time = sms.groupby(['uid'])['start_time'].agg(['std','max','min','median','mean','sum']).add_prefix('sms_gap_time_').reset_index()
sms_last_start_gap_time = sms.groupby(['uid'])['start_time'].agg({'unique_count': lambda x: max(pd.unique((x//10000)%100)),'count':'count'}).add_prefix('sms_last_start_gap_time_').reset_index()
sms_early_start_gap_time = sms.groupby(['uid'])['start_time'].agg({'unique_count': lambda x: min(pd.unique((x//10000)%100)),'count':'count'}).add_prefix('sms_early_start_gap_time_').reset_index()
sms_hour_count= sms.groupby(['uid'])['start_time'].agg({'unique_count': lambda x: len(pd.unique((x//10000)%100)),'count':'count'}).add_prefix('sms_hour_count_').reset_index()

### 网站/APP记录

In [188]:
wa_name = wa.groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('wa_name_').reset_index()
visit_cnt = wa.groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_').reset_index()

visit_dura = wa.groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_').reset_index()


up_flow = wa.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_').reset_index()

down_flow = wa.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_').reset_index()

In [189]:
wa_name_1 = wa[wa.wa_type==1.0].groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('wa_name_1_').reset_index()
visit_cnt_1 = wa[wa.wa_type==1.0].groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_1_').reset_index()

visit_dura_1 = wa[wa.wa_type==1.0].groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_1_').reset_index()


up_flow_1 = wa[wa.wa_type==1.0].groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_1_').reset_index()

down_flow_1 = wa[wa.wa_type==1.0].groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_1_').reset_index()

In [190]:
wa_name_0 = wa[wa.wa_type==0.0].groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('wa_name_0_').reset_index()
visit_cnt_0 = wa[wa.wa_type==0.0].groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_0_').reset_index()

visit_dura_0 = wa[wa.wa_type==0.0].groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_0_').reset_index()


up_flow_0 = wa[wa.wa_type==0.0].groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_0_').reset_index()

down_flow_0 = wa[wa.wa_type==0.0].groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_0_').reset_index()

In [132]:
# feature = [voice_opp_num,voice_opp_head,voice_start_time,voice_end_time,voice_opp_len,voice_call_type,voice_in_out,sms_opp_num,sms_opp_head,sms_opp_len,sms_start_time,sms_in_out,wa_name,visit_cnt,visit_dura,up_flow,
#            down_flow,wa_name_1,visit_cnt_1,visit_dura_1,up_flow_1,down_flow_1,wa_name_0,visit_cnt_0,visit_dura_0,up_flow_0,down_flow_0,
#            voice_start_hour_count,voice_end_hour_count,voice_start_day_count,voice_end_day_count,voice_early_hour,voice_late_hour,voice_end_early_hour,voice_end_late_hour,
#            sms_early_start_gap_time,sms_last_start_gap_time,sms_hour_count]

In [127]:
# feature = [voice_opp_num,voice_opp_head,voice_start_time,voice_end_time,voice_opp_len,voice_call_type,voice_in_out,sms_opp_num,sms_opp_head,sms_opp_len,sms_in_out,wa_name,visit_cnt,visit_dura,up_flow,
#            down_flow,wa_name_1,visit_cnt_1,visit_dura_1,up_flow_1,down_flow_1,wa_name_0,visit_cnt_0,visit_dura_0,up_flow_0,down_flow_0,voice_gap_time,voice_last_gap_time
#           ,sms_last_start_gap_time,sms_hour_count,voice_start_hour_count]

In [134]:
# train_feature = uid_train
# for feat in feature:
#     train_feature=pd.merge(train_feature,feat,how='left',on='uid')

In [129]:
# test_feature = uid_test
# for feat in feature:
#     test_feature=pd.merge(test_feature,feat,how='left',on='uid')

In [75]:
# train_feature.to_csv('train_featureV4.csv',index=None)
# test_feature.to_csv('test_featureV4.csv',index=None)

In [178]:
# dtrain = lgb.Dataset(train_feature.drop(['uid','label'],axis=1),label=train.label)
# dtest = lgb.Dataset(test_feature.drop(['uid'],axis=1))

In [194]:
feature = [voice_opp_num,voice_gap_time,voice_call_type_opp_num_unique,voice_in_out_opp_num_unique,voice_opp_head,voice_opp_len,voice_call_type,voice_in_out,sms_opp_num,sms_opp_head,sms_opp_len,sms_in_out,wa_name,visit_cnt,visit_dura,up_flow,
#            down_flow,wa_name_1,visit_cnt_1,visit_dura_1,up_flow_1,down_flow_1,wa_name_0,visit_cnt_0,visit_dura_0,up_flow_0,down_flow_0,
           voice_start_hour_count,voice_end_hour_count,voice_start_day_count,voice_end_day_count,voice_early_hour,voice_late_hour,voice_end_early_hour,voice_end_late_hour,
#            sms_early_start_gap_time,sms_last_start_gap_time,sms_hour_count
          ]
train_feature = uid_train
for feat in feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')
test_feature = uid_test
for feat in feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')
dtrain = lgb.Dataset(train_feature.drop(['uid','label'],axis=1),label=train.label)
dtest = lgb.Dataset(test_feature.drop(['uid'],axis=1))

In [206]:
# train_feature.to_csv('train_featureV4.csv',index=None)
# test_feature.to_csv('test_featureV4.csv',index=None)